In [27]:
import statistics
import matplotlib.pyplot as plt
import pymysql
import config
import transformations
import ml
from sklearn.model_selection import KFold
import pandas as pd

In [20]:
conn = pymysql.connect(config.host, user=config.username,port=config.port,
                           passwd=config.password)

#gather all historical data to build model
RideWaits = pd.read_sql_query("call DisneyDB.RideWaitQuery", conn)

#transform data for model bulding
RideWaits = transformations.transformData(RideWaits)

C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["MagicHourType"] = pd.Categorical(RideWaits["MagicHourType"])
C:\Users\chrisA\Documents\DisneyWaitTimes\DisneyWaits\src\transformations.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  RideWaits["TimeSinceRideOpen"] = (RideWaits["Date"] - RideWaits["OpeningDate"]).dt.days


In [21]:
RideWaits.head()

,RideId,Date,Time,Wait,Name,OpeningDate,Tier,Location,IntellectualProp,Status,...,Weekend,CharacterExperience,inEMH,validTime,EMHDay,TimeSinceOpen,TimeSinceMidday,MagicHourType,MinutesSinceOpen,TimeSinceRideOpen
0,0,2018-05-07,12:30:00,35,Astro Orbiter,1995-02-25,minor_attraction,Tomorrowland,None,1,...,0,0,0,1,1,3,2,Night,210.0,8472
1,2,2018-05-07,12:30:00,45,Big Thunder Mountain Railroad,1980-09-23,headliner,Frontierland,None,1,...,0,0,0,1,1,3,2,Night,210.0,13740
2,3,2018-05-07,12:30:00,45,Buzz Lightyears Space Ranger Spin,1998-10-07,minor_attraction,Tomorrowland,Pixar,1,...,0,0,0,1,1,3,2,Night,210.0,7152
3,6,2018-05-07,12:30:00,40,Dumbo the Flying Elephant,1971-10-01,minor_attraction,Fantasyland,AnimatedClassic,1,...,0,0,0,1,1,3,2,Night,210.0,17020
4,7,2018-05-07,12:30:00,25,Enchanted Tales with Belle,2012-12-06,minor_attraction,Fantasyland,AnimatedClassic,1,...,0,0,0,1,1,3,2,Night,210.0,1978


In [22]:
keyFeatures = ["Name","MagicHourType",
                "Tier", "IntellectualProp",
                "SimpleStatus", "ParkName",
                "DayOfWeek", "Weekend", "TimeSinceOpen", "MinutesSinceOpen",
                "CharacterExperience", "TimeSinceMidday",
                "inEMH", "EMHDay"]
newModel = ml.buildModel(RideWaits, keyFeatures, "Wait")

Fold 1
RMSE: 4.46172967964
Correlation: (0.98864342113312376, 0.0)
Accuracy: 0.945013756614
Mean Error: 2.92966379729
Median Error: 1.85949514991
-------------------------
Fold 2
RMSE: 7.03497623707
Correlation: (0.97363677113571956, 0.0)
Accuracy: 0.939251625882
Mean Error: 3.78226183067
Median Error: 1.81204264871
-------------------------
Fold 3
RMSE: 14.4372916564
Correlation: (0.87014216643796305, 0.0)
Accuracy: 0.749334904101
Mean Error: 9.28618081174
Median Error: 5.66879739859
-------------------------
Fold 4
RMSE: 16.4302278353
Correlation: (0.82920242676640976, 0.0)
Accuracy: 0.759914273117
Mean Error: 10.3774965216
Median Error: 5.60039902998
-------------------------
Fold 5
RMSE: 14.9965181288
Correlation: (0.84318287589328478, 0.0)
Accuracy: 0.72456100823
Mean Error: 10.0940798454
Median Error: 6.61706880311
-------------------------
Fold 6
RMSE: 17.4790698531
Correlation: (0.82104743137737446, 0.0)
Accuracy: 0.765818758573
Mean Error: 10.9250771869
Median Error: 6.0680160

In [26]:
RideWaits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69656 entries, 0 to 72086
Data columns (total 31 columns):
RideId                 69656 non-null int8
Date                   69656 non-null datetime64[ns]
Time                   69656 non-null object
Wait                   69656 non-null int64
Name                   69656 non-null int8
OpeningDate            69656 non-null datetime64[ns]
Tier                   69656 non-null int8
Location               69656 non-null object
IntellectualProp       69656 non-null int8
Status                 69656 non-null int8
Temperature            69656 non-null float64
CloudCover             69656 non-null float64
SimpleStatus           69656 non-null int8
RainAccumulation       69656 non-null float64
ParkId                 69656 non-null int8
ParkOpen               69656 non-null object
ParkClose              69656 non-null object
EMHOpen                24827 non-null object
EMHClose               24827 non-null object
ParkName               69656 non